In [1]:
import torch
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
from pathlib import Path
import albumentations as A
import timm
from albumentations.pytorch import ToTensorV2
import torch.nn as nn


from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, average_precision_score
import seaborn as sns
import matplotlib.pyplot as plt


import random
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)


c:\Users\ACER NITRO\anaconda3\envs\cuda\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.4 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
class SkinCancerModel(nn.Module):
    def __init__(self, num_classes=1):
        super(SkinCancerModel, self).__init__()
        self.backbone = timm.create_model("swin_tiny_patch4_window7_224", pretrained=True, num_classes=0)
        feature_dim = self.backbone.num_features
        
        for param in self.backbone.parameters():
            param.requires_grad = False
        for param in list(self.backbone.layers[-2:].parameters()):
            param.requires_grad = True
        
        self.classifier = nn.Sequential(
            nn.Linear(feature_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1)
        )
    
    def forward(self, image):
        image_features = self.backbone(image)
        output = self.classifier(image_features)
        return output.squeeze(1)


In [3]:
def load_model(model: torch.nn.Module, model_path: str, device: torch.device):
    model_path = Path(model_path)
    if not model_path.exists():
        raise FileNotFoundError(f"Model file not found: {model_path}")
    print(f"Loading model from: {model_path}")
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model


In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

tta_transforms = [
    A.HorizontalFlip(p=1),
    A.RandomBrightnessContrast(p=1, brightness_limit=0.5, contrast_limit=0.4),
    A.ElasticTransform(p=1),
    A.RandomScale(p=1, scale_limit=0.2),
    A.Rotate(p=1, limit=45)
]

def apply_tta(image, transform_fn):
    return transform_fn(image=image)['image']


In [ ]:
def tta_predict(model, image_path, augmentations, device):
    original_image = Image.open(image_path).convert("RGB")
    tta_preds = []
    model.eval()
    
    with torch.no_grad():
        transformed_image = transform(original_image).unsqueeze(0).to(device)
        original_pred = model(transformed_image)
        tta_preds.append(torch.sigmoid(original_pred))

        for aug in augmentations:
            aug_image_np = apply_tta(np.array(original_image), aug)
            aug_image = Image.fromarray(aug_image_np)
            transformed_aug_image = transform(aug_image).unsqueeze(0).to(device)
            aug_pred = model(transformed_aug_image)
            tta_preds.append(torch.sigmoid(aug_pred))

    return torch.mean(torch.stack(tta_preds), dim=0)

# Function to evaluate model and compute accuracy

def evaluate_model(model, test_metadata, device, skin_tone_column=None):
    if skin_tone_column:
        skin_tone_df = test_metadata[test_metadata[skin_tone_column] == 1].reset_index(drop=True)
    else:
        skin_tone_df = test_metadata.reset_index(drop=True)

    y_true, y_pred_classes, y_pred_probs = [], [], []

    for _, row in skin_tone_df.iterrows():
        image_path = row["DDI_path"]
        true_label = row["malignant"]

        final_pred = tta_predict(model, image_path, tta_transforms, device)
        predicted_prob = final_pred.item()
        predicted_class = 1 if predicted_prob >= 0.5 else 0  # Apply threshold at 0.5

        y_true.append(true_label)
        y_pred_probs.append(predicted_prob)
        y_pred_classes.append(predicted_class)

    # Compute metrics
    accuracy = accuracy_score(y_true, y_pred_classes)
    auc = roc_auc_score(y_true, y_pred_probs) if len(set(y_true)) > 1 else None

    return accuracy, auc


In [ ]:


# Initialize DataFrame to store results
df = pd.DataFrame(columns=["tta_augmentation", "model_name", "auc", "auc_12", "auc_34", "auc_56", 
                           "accuracy", "accuracy_12", "accuracy_34", "accuracy_56"])

# Load model
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SkinCancerModel().to(DEVICE)
model = load_model(model, model_path="models/image_only_skin_cancer.pth", device=DEVICE)
model.eval()

# Load test metadata
test_metadata = pd.read_csv("data/test_metadata.csv")

# Global evaluation
global_acc, global_auc = evaluate_model(model, test_metadata, DEVICE)
print("\nGlobal Evaluation:")
print(f"Accuracy: {global_acc:.4f}")
print(f"AUC: {global_auc:.4f}" if global_auc is not None else "AUC: Not computed (only one class present)")

# Evaluate per skin tone group
skin_tone_columns = ["skin_tone_12", "skin_tone_34", "skin_tone_56"]
accuracies, aucs = {}, {}

for skin_tone_col in skin_tone_columns:
    acc, auc = evaluate_model(model, test_metadata, DEVICE, skin_tone_column=skin_tone_col)
    accuracies[skin_tone_col] = acc
    aucs[skin_tone_col] = auc

    print(f"\nEvaluation for {skin_tone_col}:")
    print(f"Accuracy: {acc:.4f}")
    print(f"AUC: {auc:.4f}" if auc is not None else "AUC: Not computed (only one class present)")

# Store results in DataFrame
df.loc[len(df)] = {
    "tta_augmentation": 1,
    "model_name": "swinTransformers",
    "auc": global_auc,
    "auc_12": aucs["skin_tone_12"],
    "auc_34": aucs["skin_tone_34"],
    "auc_56": aucs["skin_tone_56"],
    "accuracy": global_acc,
    "accuracy_12": accuracies["skin_tone_12"],
    "accuracy_34": accuracies["skin_tone_34"],
    "accuracy_56": accuracies["skin_tone_56"]
}




In [6]:
df


,tta_augmentation,model_name,auc,auc_12,auc_34,auc_56,accuracy,accuracy_12,accuracy_34,accuracy_56
0,1,swinTransformers,0.869333,0.858824,0.8125,0.970588,0.830769,0.863636,0.772727,0.904762
